In [64]:
import nltk
from nltk.util import ngrams
import re
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download("stopwords") # поддерживает удаление стоп-слов
nltk.download('punkt') # делит текст на список предложений
nltk.download('wordnet') # проводит лемматизацию
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrsipko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandrsipko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aleksandrsipko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [65]:
import pandas as pd
import numpy as np
import nltk
import re
import csv
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import pymorphy2
import seaborn as sns
sns.set_style("darkgrid")
from nltk.stem.snowball import SnowballStemmer

In [66]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrsipko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Функции

In [67]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('russian'))

def csv_to_list(arr):
    arr_list = []
    for row in arr:
        arr_list.append(list_to_str(row))
    return arr_list

def list_to_str(arr):
    str_ = ''
    for rec in arr:
        str_+=rec
    return str_

def df_preprocess(text):  
    reg = re.compile('[^а-яА-яa-zA-Z ]') #
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text)
    text = re.sub('@[^\s]+', 'пользователь', text)
    text = reg.sub(' ', text)
    
    # Лемматизация
    #morph = pymorphy2.MorphAnalyzer()
    #text =[morph.parse(word)[0].normal_form for word in text.split()]

    # Стемминг
    # stemmer = SnowballStemmer("russian")
    # text =[stemmer.stem(word) for word in text.split()]

    # Стемминг + удаление стоп слов
    stemmer = SnowballStemmer("russian")
    #text =[stemmer.stem(word) for word in text.split() if word not in stopWords]
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopWords])

    return text

## Считываем данные

In [68]:
positive_recalls = csv_to_list(csv.reader(codecs.open('positive.csv', 'rU', 'utf-8', errors='ignore')))
negative_recalls = csv_to_list(csv.reader(codecs.open('negative.csv', 'rU', 'utf-8', errors='ignore')))

## Формируем датасет

In [69]:
df_positive_recalls = pd.DataFrame(positive_recalls, columns=['recall'])
df_positive_recalls['type']=1
df_positive_recalls.head(20)

,recall,type
0,idtdatetnamettextttypetreptrtwtfavtstcounttfol...,1
1,4089066923744460801386325927pleease_shut_up@fi...,1
2,"4089066926932213771386325927alinakirpichevaДа,...",1
3,4089066950839541771386325927EvgeshaReRT @Katia...,1
4,4089066953569730561386325927ikonnikova_21RT @d...,1
5,4089067614168678421386325943JumpyAlex@irina_dy...,1
6,4089067617695989761386325943JustinB94262583ну ...,1
7,4089067624364810241386325943twinkleAYORT @Spoo...,1
8,4089067641142067201386325944pycalyruhogRT @ver...,1
9,4089067646087495681386325944grishintvПоприветс...,1


In [70]:
df_negative_recalls = pd.DataFrame(negative_recalls, columns=['recall'])
df_negative_recalls['type']=0
df_negative_recalls.head()

,recall,type
0,idtdatetnamettextttypetreptrtwtfavtstcounttfol...,0
1,4089067628135792641386325944dugarchikbellkoна ...,0
2,4089068182626877441386325957nugemycejelaКоллег...,0
3,40890685851539865613863259664post21@elina_4pos...,0
4,4089069144376852481386325980PoliwakeЖелаю хоро...,0


In [71]:
# Объединяем два датафрейма вместе
df_recalls = pd.concat((df_negative_recalls, df_positive_recalls),axis = 0).sample(frac = 1.0) # объединяем и перемешиваем
df_recalls.index = range(0,len(df_recalls))
df_recalls.head()

,recall,type
0,4093270101591326721386426138little_tyranобожаю...,1
1,4133334806896435211387381355Katya__Fox@ao_boss...,0
2,4209302944942899201389192577JDuryaginaRT @Ulya...,0
3,4227553040075530241389627693ginaeff@37taxi @Op...,0
4,4102945175986585601386656810Kate_Shusha@staska...,1


### Удалим стоп слова

In [72]:
stopWords = set(stopwords.words('russian'))

### Очистка текста

In [73]:
%time df_recalls['recall'] = df_recalls['recall'].apply(df_preprocess)

CPU times: user 1min 58s, sys: 1.61 s, total: 1min 59s
Wall time: 2min 1s


In [74]:
df_recalls.head()

,recall,type
0,little тыранобожа люд картав разумеет фраз век...,1
1,katya фоxпользовател сегодн обижа,0
2,jduryaginart пользовател посмотрел хулига зеле...,0
3,гинаеффпользовател пользовател херас эт скольк...,0
4,kate шушапользовател кажет станов бессмен лиде...,1


In [75]:
df_recalls.shape

(226836, 2)

### TRAIN/test split

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_recalls['recall'], df_recalls['type'], test_size=.15, random_state=42)

In [77]:
type(X_train)

pandas.core.series.Series

In [78]:
print(len(X_train), len(X_test))

192810 34026


In [79]:
X_train[100]

'конторусовникт независим захотел отвезт син сотк'

### BOW

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X_train_BOW = vectorizer.fit_transform(X_train)
X_test_BOW = vectorizer.transform(X_test)

In [81]:
print(X_train_BOW.shape, X_test_BOW.shape)

(192810, 238761) (34026, 238761)


In [82]:
X_train.iloc[200]

'fitcurves ыоукоманд центральн офис встреч владельц клуб fitcurves киев стран должн знат сво геро сайт'

In [83]:
# Векторное представление
X_train_BOW[200]

<1x238761 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

### TF-IDF

In [84]:
from sklearn.feature_extraction.text import  TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_TFIDF = vectorizer.fit_transform(X_train)
X_test_TFIDF = vectorizer.transform(X_test)

In [85]:
print(X_train_TFIDF.shape, X_test_TFIDF.shape)

(192810, 238761) (34026, 238761)


In [86]:
# Векторное представление
X_train_TFIDF[200]

<1x238761 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

### Строим простейшую модель
На данных BOW

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [88]:
# обучаем классификатор
%time clf = LogisticRegression(random_state=0).fit(X_train_BOW, y_train)

CPU times: user 13.9 s, sys: 28.2 s, total: 42.1 s
Wall time: 9.39 s


/Users/aleksandrsipko/scripts_python/text_preparations/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [89]:
# вычисляем предсказания
y_predict_BOW = clf.predict(X_test_BOW)

In [90]:
# вычисляем метрику accuracy
accuracy_score(y_predict_BOW, y_test)

0.7382589784282607

### На данных TF-IDF

In [91]:
%time clf = LogisticRegression(random_state=43).fit(X_train_TFIDF, y_train)

CPU times: user 15 s, sys: 32 s, total: 47 s
Wall time: 9.89 s


/Users/aleksandrsipko/scripts_python/text_preparations/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [92]:
y_predict_TFIDF = clf.predict(X_test_TFIDF)
accuracy_score(y_predict_TFIDF, y_test)

0.7341150884617645

### На данных BOW с биграммами

In [93]:
#построение ngramm для данных
vectorizer = CountVectorizer(ngram_range=(1, 2))

X_train_BOW_bi = vectorizer.fit_transform(X_train)
X_test_BOW_bi = vectorizer.transform(X_test)

print(X_train_BOW_bi.shape, X_test_BOW_bi.shape)
#-----------------------------------------------
clf = LogisticRegression(random_state=0).fit(X_train_BOW_bi, y_train)
#-----------------------------------------------
y_predict_BOW_bi = clf.predict(X_test_BOW_bi)
#-----------------------------------------------
accuracy_score(y_predict_BOW_bi, y_test)

(192810, 1191752) (34026, 1191752)


/Users/aleksandrsipko/scripts_python/text_preparations/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7533062951860342